# Movies Recommend System

レビュー類似度を用いた Movies Recommend System の開発

使用する映画レビューサイトは　http://www.jtnews.jp/index.html

### 構成

1. スクレイピング/SQL
2. レビューの形態素解析
3. 特徴量行列の作成
4. モデル作成

### 1. スクレイピング/SQL

In [1]:
from bs4 import BeautifulSoup
import requests

In [2]:
import pandas as pd
from pandas import Series,DataFrame

In [3]:
import sqlite3

In [4]:
con = sqlite3.connect("movies.db")

In [5]:
# 関数を作って、SQL文の実行結果をDataFrameにして返します。
def sql_to_df(sql_query):

    # read_sqlの引数に、SQL文とデータベースへのConnectionを渡します。
    df = pd.read_sql(sql_query, con)

    # 結果のDataFrameを返します。
    return df

In [6]:
import time
import re

In [8]:
i = 0

i_id = []
all_title = []
data = []

for num in range(10,90):
    c_url = []
    
    time.sleep(3)

    url = 'http://www.jtnews.jp/cgi-bin/review.cgi?TITLE_NO=' + str(num)
    
    result = requests.get(url)
    c = result.content

    soup = BeautifulSoup(c, "lxml")
    
    title = soup.find("title")
    title_text = title.find(text=True)

    for a in soup.find_all('a'):
        text = "PAGE_NO="
        pa = a.get('href')
        if re.match('.*PAGE_NO=[^1]', pa):
            pa = 'http://www.jtnews.jp/cgi-bin/' + pa
            if pa not in c_url:
                c_url.append(pa)
    
    c_url.insert(0, url)

    for item in c_url:
        
        time.sleep(3)
        
        print(item)
        
        result = requests.get(item)
        c = result.content

        soup = BeautifulSoup(c, "lxml")

        reviews = soup.find_all('table', {'class':'REV_MAIN_DATA'}) 
        
        # テーブルから行をすべて探し出します。
        rows = reviews[0].find_all('tr')
        
        # 行から、それぞれのcellを取り出して画面に表示しつつ、dataに格納します。
        for tr in rows:
            cols = tr.find_all('td')
            
            # textを探し出します。
            for td in cols:
                text = td.find_all(text=True)
                data_rows_renketsu = ""
                
                for n in range(len(text)):
                    data_rows_renketsu += text[n]
                    
                i += 1    
                
                #print("i_id : ", i)
                #print("title_text : ", title_text)                
                #print("data_rows_renketsu : ", data_rows_renketsu)
            
                i_id.append(i)
                all_title.append(title_text)            
                data.append(data_rows_renketsu)  

http://www.jtnews.jp/cgi-bin/review.cgi?TITLE_NO=10
i_id :  1
i_id :  2
i_id :  3
i_id :  4
i_id :  5
i_id :  6
i_id :  7
i_id :  8
i_id :  9
i_id :  10
i_id :  11
i_id :  12
i_id :  13
i_id :  14
i_id :  15
i_id :  16
i_id :  17
i_id :  18
i_id :  19
i_id :  20
http://www.jtnews.jp/cgi-bin/review.cgi?BOOK_MARK=0&DSP_KIND=0&NETA=0&PAGE_NO=2&POINT=0&RTN=1&SORT=0&TITLE_NO=10
i_id :  21
i_id :  22
i_id :  23
i_id :  24
i_id :  25
i_id :  26
i_id :  27
i_id :  28
i_id :  29
i_id :  30
i_id :  31
i_id :  32
i_id :  33
i_id :  34
i_id :  35
http://www.jtnews.jp/cgi-bin/review.cgi?TITLE_NO=11
i_id :  36
i_id :  37
i_id :  38
i_id :  39
i_id :  40
http://www.jtnews.jp/cgi-bin/review.cgi?TITLE_NO=12
i_id :  41
i_id :  42
i_id :  43
i_id :  44
i_id :  45
i_id :  46
i_id :  47
i_id :  48
i_id :  49
i_id :  50
i_id :  51
i_id :  52
i_id :  53
i_id :  54
i_id :  55
i_id :  56
i_id :  57
i_id :  58
i_id :  59
i_id :  60
http://www.jtnews.jp/cgi-bin/review.cgi?BOOK_MARK=0&DSP_KIND=0&NETA=0&PAGE_NO=2&

データはSQLで扱いやすいように、DataFrame型に変えていきます。

・data = [] / list

・data = Series(data)

・legislative_df = pd.concat([data, axis=1) / DataFrame

In [9]:
i_id =Series(i_id)
all_title = Series(all_title)
data = Series(data)

In [10]:
# 連結してDataFrameにします。
reviews_df = pd.concat([i_id, all_title, data],axis=1)
reviews_df.columns=['id', 'title','review']

In [11]:
reviews_df

,id,title,review
0,1,愛は静けさの中に - みんなのシネマレビュー,35．《ネタバレ》 ヒロインのマーリー・マトリンは、イザベル・アジャーニみたいに美人だけど、...
1,2,愛は静けさの中に - みんなのシネマレビュー,34．昔ながらのよくある正統派のラブストーリーだが、聾学校という舞台背景が設定の妙味に。ヒロ...
2,3,愛は静けさの中に - みんなのシネマレビュー,33．耳が聞こえる男と耳が聞こえない女は、どうすれば一つの同じ世界に生きることができるだろう...
3,4,愛は静けさの中に - みんなのシネマレビュー,32．生徒や恋人、聾唖者との関わりを重視したドラマだと思っていたが、よくある一組のカップルの...
4,5,愛は静けさの中に - みんなのシネマレビュー,31．《ネタバレ》 二人の恋のゆくえより、生徒とのやりとりのほうが面白かった。生徒達はかわい...
5,6,愛は静けさの中に - みんなのシネマレビュー,30．《ネタバレ》 マーリー・マトリン美しい… 賞受賞も納得の内容で演出面も素晴らしい！ ...
6,7,愛は静けさの中に - みんなのシネマレビュー,29．教師役として抑えた演技が光るウィリアム・ハートと、この作品でオスカーを獲得した女優マー...
7,8,愛は静けさの中に - みんなのシネマレビュー,28．自分が好きだった人と、ヒロインが同じ髪型（「フラッシュダンス」のジェニファービールスと...
8,9,愛は静けさの中に - みんなのシネマレビュー,27．こんな純愛ドラマがあったなぁとか思いながら鑑賞しました。ストーリーには凝ったところもな...
9,10,愛は静けさの中に - みんなのシネマレビュー,26．ウィリアム・ハートとマーリー・マトリン、この二人の演技がすごい。特にマーリー・マトリン...


In [12]:
hiku_df = reviews_df.ix[reviews_df['review'].str.contains('"')]

In [13]:
hiku_df

,id,title,review
21,22,愛は静けさの中に - みんなのシネマレビュー,14． 原作はマーク・メドフの舞台劇「小さき神の子等」。ヒロインのマーリー・マトリンは舞台か...
480,481,アビス - みんなのシネマレビュー,"129．Through a series named as ""See It Big! Sci..."
818,819,アポロ１３ - みんなのシネマレビュー,"252．""アポロ13号""。以前、原作を本で読んでいたく感動しました。さて映画はというと、良か..."
912,913,アマデウス - みんなのシネマレビュー,"369．空想のストーリーとしてはなかなか面白いですよ。真実など誰も知らないと前提して。""だっ..."
1163,1164,アラジン - みんなのシネマレビュー,"114．《ネタバレ》 作品のテーマは""自由""。映画には３人の自由を求めるキャラクターがいる。..."
1277,1278,アリス(1990) - みんなのシネマレビュー,"14．""金満夫人の憂鬱な日々""といった内容の女性映画。セレブにはセレブなりの悩みや不満がある..."
1295,1296,ある貴婦人の肖像 - みんなのシネマレビュー,26．私の大好きなマルコビッチ様が、ただの悪役だったのがショック（>＿<）´´途中「危険な関...
1579,1580,イージー・ライダー - みんなのシネマレビュー,"156．《ネタバレ》 ステッペン・ウルフの""Born to be Wild""の曲がいいですね..."
1587,1588,イージー・ライダー - みんなのシネマレビュー,148．観る前の印象としては、「世間じゃ自由・平等などと騒いじゃいるが、俺たちにゃそんなこと...
1607,1608,イージー・ライダー - みんなのシネマレビュー,"128．音楽がよかった。""Born To Be Wild""が流れる冒頭のシーンは鳥肌もんでし..."


In [16]:
reviews_df = reviews_df.drop([21, 480,818,912,1163,1277,1295,1579,1587,1607,2306,2550,2784,4077,4249,4253,4263,4827,4878,4946,5483,5490,6436,6957,7148])

In [17]:
reviews_df.ix[reviews_df['review'].str.contains('"')]

,id,title,review


次はレビューを読みだして形態素解析を行っていく。

### 2. レビューの形態素解析

In [18]:
import MeCab

In [19]:
mecab = MeCab.Tagger ('-d /usr/local/lib/mecab/dic/mecab-ipadic-neologd')

##### 事前に作成しているテーブル

CREATE TABLE cnt_table(
    id int,
    title char(50),
    word char(30),
    num int
);

In [20]:
for item in range(len(reviews_df)):
    time.sleep(0.1)    
    i_id = reviews_df.iloc[item, 0]
    title = reviews_df.iloc[item,1]
    review = reviews_df.iloc[item,2]
    print(i_id)
    #print(title)
    #print(review)
    
    text = review
    mecab.parse('')#文字列がGCされるのを防ぐ
    node = mecab.parseToNode(text)
    while node:
        #単語を取得
        word = node.surface
        #品詞を取得
        pos = node.feature.split(",")[1]
        #print('{}'.format(word))
        #print('{0} , {1}'.format(word, pos))
        
        #次の単語に進める
        node = node.next
        
        sql_query = '''
        INSERT INTO cnt_table(id, title, word, num) values({i_id}, "{title}", "{word}", 1)
        '''.format(i_id=i_id, title=title, word=word)
        
        con.execute(sql_query)
        

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278

In [137]:
sql_query = '''
SELECT * FROM cnt_table
'''
words_df = sql_to_df(sql_query)

In [138]:
words_df.tail()

,id,title,word,num


In [ ]:
#con.close()

### 3. 特徴量行列の作成

In [21]:
i = 1
words_cnt = []

In [22]:
#The first step —————————————————————————————————————————————————
#全映画で使われたwordをとりだす。
sql_query = '''
SELECT word FROM cnt_table WHERE word not like '%''%' GROUP BY word ORDER BY sum(num) desc
'''
words_used_df = sql_to_df(sql_query)

In [23]:
words_used_df.tail()

,word
29313,ﾜﾗﾀ
29314,ﾟ
29315,￣
29316,￣)
29317,￣*


In [24]:
#The second step ————————————————————————————————————————————————
num = 0
query_1 = '''SELECT id, '''
for item in range(len(words_used_df)):
    #レビューIDごとに出現しているwordでカウントする。
    query_1 += '''SUM(CASE WHEN word = '{word}' THEN num ELSE 0 END) AS a{num}, '''.format(word=words_used_df.iloc[item, 0], num=num)
        
    num += 1
    
    if num==1900:
        break
    
query_1 += '''title FROM cnt_table GROUP BY id, title'''

In [25]:
#The second step ————————————————————————————————————————————————
num = 0
query_2 = '''SELECT id, '''
for item in range(len(words_used_df)):
    num += 1
    if num >= 1900:
        #レビューIDごとに出現しているwordでカウントする。
        query_2 += '''SUM(CASE WHEN word = '{word}' THEN num ELSE 0 END) AS a{num}, '''.format(word=words_used_df.iloc[item, 0], num=num)
        
        if num==3800:
            break
    
query_2 += '''title FROM cnt_table GROUP BY id, title'''

In [26]:
#The second step ————————————————————————————————————————————————
num = 0
query_3 = '''SELECT id, '''
for item in range(len(words_used_df)):
    num += 1
    if num >= 3800:
        #レビューIDごとに出現しているwordでカウントする。
        query_3 += '''SUM(CASE WHEN word = '{word}' THEN num ELSE 0 END) AS a{num}, '''.format(word=words_used_df.iloc[item, 0], num=num)
        
        if num==5700:
            break
    
query_3 += '''title FROM cnt_table GROUP BY id, title'''

In [35]:
#The second step ————————————————————————————————————————————————
num = 0
query_4 = '''SELECT id, '''
for item in range(len(words_used_df)):
    num += 1
    if num >= 5700:
        #レビューIDごとに出現しているwordでカウントする。
        query_4 += '''SUM(CASE WHEN word = '{word}' THEN num ELSE 0 END) AS a{num}, '''.format(word=words_used_df.iloc[item, 0], num=num)
        
        if num==7600:
            break
    
query_4 += '''title FROM cnt_table GROUP BY id, title'''

In [37]:
#The second step ————————————————————————————————————————————————
num = 0
query_5 = '''SELECT id, '''
for item in range(len(words_used_df)):
    num += 1
    if num >= 7600:
        #レビューIDごとに出現しているwordでカウントする。
        query_5 += '''SUM(CASE WHEN word = '{word}' THEN num ELSE 0 END) AS a{num}, '''.format(word=words_used_df.iloc[item, 0], num=num)
        
        if num==9500:
            break
    
query_5 += '''title FROM cnt_table GROUP BY id, title'''

In [38]:
#The second step ————————————————————————————————————————————————
num = 0
query_6 = '''SELECT id, '''
for item in range(len(words_used_df)):
    num += 1
    if num >= 9500:
        #レビューIDごとに出現しているwordでカウントする。
        query_6 += '''SUM(CASE WHEN word = '{word}' THEN num ELSE 0 END) AS a{num}, '''.format(word=words_used_df.iloc[item, 0], num=num)
        
        if num==11400:
            break
    
query_6 += '''title FROM cnt_table GROUP BY id, title'''

In [39]:
#The second step ————————————————————————————————————————————————
num = 0
query_7 = '''SELECT id, '''
for item in range(len(words_used_df)):
    num += 1
    if num >= 11400:
        #レビューIDごとに出現しているwordでカウントする。
        query_7 += '''SUM(CASE WHEN word = '{word}' THEN num ELSE 0 END) AS a{num}, '''.format(word=words_used_df.iloc[item, 0], num=num)
        
        if num==13300:
            break
    
query_7 += '''title FROM cnt_table GROUP BY id, title'''

In [41]:
#The second step ————————————————————————————————————————————————
num = 0
query_9 = '''SELECT id, '''
for item in range(len(words_used_df)):
    num += 1
    if num >= 13300:
        #レビューIDごとに出現しているwordでカウントする。
        query_9 += '''SUM(CASE WHEN word = '{word}' THEN num ELSE 0 END) AS a{num}, '''.format(word=words_used_df.iloc[item, 0], num=num)
        
        if num==15200:
            break
    
query_9 += '''title FROM cnt_table GROUP BY id, title'''

In [47]:
#The second step ————————————————————————————————————————————————
num = 0
query_10 = '''SELECT id, '''
for item in range(len(words_used_df)):
    num += 1
    if num >= 15200:
        #レビューIDごとに出現しているwordでカウントする。
        query_10 += '''SUM(CASE WHEN word = '{word}' THEN num ELSE 0 END) AS a{num}, '''.format(word=words_used_df.iloc[item, 0], num=num)
        
        if num==17100:
            break
    
query_10 += '''title FROM cnt_table GROUP BY id, title'''

In [48]:
#The second step ————————————————————————————————————————————————
num = 0
query_11 = '''SELECT id, '''
for item in range(len(words_used_df)):
    num += 1
    if num >= 17100:
        #レビューIDごとに出現しているwordでカウントする。
        query_11 += '''SUM(CASE WHEN word = '{word}' THEN num ELSE 0 END) AS a{num}, '''.format(word=words_used_df.iloc[item, 0], num=num)
        
        if num==19000:
            break
    
query_11 += '''title FROM cnt_table GROUP BY id, title'''

In [49]:
#The second step ————————————————————————————————————————————————
num = 0
query_12 = '''SELECT id, '''
for item in range(len(words_used_df)):
    num += 1
    if num >= 19000:
        #レビューIDごとに出現しているwordでカウントする。
        query_12 += '''SUM(CASE WHEN word = '{word}' THEN num ELSE 0 END) AS a{num}, '''.format(word=words_used_df.iloc[item, 0], num=num)
        
        if num==20900:
            break
    
query_12 += '''title FROM cnt_table GROUP BY id, title'''

In [50]:
#The second step ————————————————————————————————————————————————
num = 0
query_13 = '''SELECT id, '''
for item in range(len(words_used_df)):
    num += 1
    if num >= 20900:
        #レビューIDごとに出現しているwordでカウントする。
        query_13 += '''SUM(CASE WHEN word = '{word}' THEN num ELSE 0 END) AS a{num}, '''.format(word=words_used_df.iloc[item, 0], num=num)
        
        if num==22800:
            break
    
query_13 += '''title FROM cnt_table GROUP BY id, title'''

In [51]:
#The second step ————————————————————————————————————————————————
num = 0
query_14 = '''SELECT id, '''
for item in range(len(words_used_df)):
    num += 1
    if num >= 22800:
        #レビューIDごとに出現しているwordでカウントする。
        query_14 += '''SUM(CASE WHEN word = '{word}' THEN num ELSE 0 END) AS a{num}, '''.format(word=words_used_df.iloc[item, 0], num=num)
        
        if num==24700:
            break
    
query_14 += '''title FROM cnt_table GROUP BY id, title'''

In [52]:
#The second step ————————————————————————————————————————————————
num = 0
query_15 = '''SELECT id, '''
for item in range(len(words_used_df)):
    num += 1
    if num >= 24700:
        #レビューIDごとに出現しているwordでカウントする。
        query_15 += '''SUM(CASE WHEN word = '{word}' THEN num ELSE 0 END) AS a{num}, '''.format(word=words_used_df.iloc[item, 0], num=num)
        
        if num==26600:
            break
    
query_15 += '''title FROM cnt_table GROUP BY id, title'''

In [53]:
#The second step ————————————————————————————————————————————————
num = 0
query_16 = '''SELECT id, '''
for item in range(len(words_used_df)):
    num += 1
    if num >= 26600:
        #レビューIDごとに出現しているwordでカウントする。
        query_16 += '''SUM(CASE WHEN word = '{word}' THEN num ELSE 0 END) AS a{num}, '''.format(word=words_used_df.iloc[item, 0], num=num)
        
        if num==28500:
            break
    
query_16 += '''title FROM cnt_table GROUP BY id, title'''

In [27]:
res_df_1 = sql_to_df(query_1)
res_df_1 = res_df_1.drop('title', axis=1)

In [30]:
res_df_2 = sql_to_df(query_2)
res_df_2 = res_df_2.drop('title', axis=1)

In [32]:
res_df_3 = sql_to_df(query_3)
res_df_3 = res_df_3.drop('title', axis=1)

In [36]:
res_df_4 = sql_to_df(query_4)
res_df_4 = res_df_4.drop('title', axis=1)

In [42]:
res_df_5 = sql_to_df(query_5)
res_df_5 = res_df_5.drop('title', axis=1)

In [43]:
res_df_6 = sql_to_df(query_6)
res_df_6 = res_df_6.drop('title', axis=1)

In [44]:
res_df_7 = sql_to_df(query_7)
res_df_7 = res_df_7.drop('title', axis=1)

In [45]:
res_df_8 = sql_to_df(query_8)
res_df_8 = res_df_8.drop('title', axis=1)

In [46]:
res_df_9 = sql_to_df(query_9)
res_df_9 = res_df_10.drop('title', axis=1)

In [55]:
res_df_10 = sql_to_df(query_10)
res_df_10 = res_df_10.drop('title', axis=1)

In [56]:
res_df_11 = sql_to_df(query_11)
res_df_11 = res_df_11.drop('title', axis=1)

In [57]:
res_df_12 = sql_to_df(query_12)
res_df_12 = res_df_12.drop('title', axis=1)

In [58]:
res_df_13 = sql_to_df(query_13)
res_df_13 = res_df_13.drop('title', axis=1)

In [59]:
res_df_14 = sql_to_df(query_14)
res_df_14 = res_df_14.drop('title', axis=1)

In [60]:
res_df_15 = sql_to_df(query_15)
res_df_15 = res_df_15.drop('title', axis=1)

In [61]:
res_df_16 = sql_to_df(query_16)
res_df_16 = res_df_16.drop('title', axis=1)

In [163]:
#con.close()

In [130]:
res_df = pd.concat([res_df_1,res_df_2,res_df_3,res_df_4,res_df_5,res_df_6,res_df_7,res_df_8,res_df_9,res_df_10,res_df_11,res_df_12,res_df_13,res_df_14,res_df_15,res_df_16], axis=1)

In [152]:
res_df = pd.concat([res_df_1,res_df_2,res_df_3,res_df_4,res_df_5,res_df_6,res_df_7], axis=1)

In [153]:
res_df

,id,a0,a1,a2,a3,a4,a5,a6,a7,a8,...,a13291,a13292,a13293,a13294,a13295,a13296,a13297,a13298,a13299,a13300
0,1,3,4,6,2,2,3,4,3,2,...,0,0,0,0,0,0,0,0,0,0
1,2,5,4,5,4,3,3,3,3,2,...,0,0,0,0,0,0,0,0,0,0
2,3,5,6,5,6,4,3,4,1,2,...,0,0,0,0,0,0,0,0,0,0
3,4,3,1,2,1,3,0,0,1,2,...,0,0,0,0,0,0,0,0,0,0
4,5,4,5,2,3,5,2,2,3,2,...,0,0,0,0,0,0,0,0,0,0
5,6,3,0,1,3,1,0,4,1,2,...,0,0,0,0,0,0,0,0,0,0
6,7,3,3,3,3,3,0,6,1,2,...,0,0,0,0,0,0,0,0,0,0
7,8,3,2,2,3,4,0,2,1,2,...,0,0,0,0,0,0,0,0,0,0
8,9,3,3,1,3,4,1,2,0,2,...,0,0,0,0,0,0,0,0,0,0
9,10,3,3,1,2,0,1,1,2,2,...,0,0,0,0,0,0,0,0,0,0


In [154]:
res_df = res_df.drop('id', axis=1)

### 4. モデル作成

ここでは、UdemyのPython実践講座の Lec80-81 を参考にしてロジスティック回帰での多値分類を行う。

In [134]:
import numpy as np
import pandas as pd
from pandas import Series,DataFrame

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

%matplotlib inline

In [135]:
from sklearn import linear_model

参考 : http://qiita.com/yshi12/items/9f3f8ae69588da9f018f

In [155]:
res_df.target = res_df['title']

In [156]:
res_numpyMatrix_target = res_df.target.as_matrix()

In [158]:
res_df.data = res_df.iloc[ : , 1:13300]

In [159]:
res_numpyMatrix_data = res_df.iloc[ : , 1:13300].as_matrix()

In [160]:
# 説明変数をXに
X = res_numpyMatrix_data

#目的変数をYに
Y = res_numpyMatrix_target

In [161]:
#pandas.DataFrameにしておきましょう。

_data = DataFrame(X)

_target = DataFrame(Y,columns=['title'])

scikit-learnを使った多クラス分類


In [162]:
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split

logreg = LogisticRegression()

# データを分割します。テストが全体の40%になるようにします。
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.4,random_state=3)

# データを使って学習します。
logreg.fit(X_train, Y_train)

ValueError: could not convert string to float: '仮面の男(1998) - みんなのシネマレビュー'

In [ ]:
# 精度を計算するのに便利なツールです。
from sklearn import metrics

# テストデータを予測します。
Y_pred = logreg.predict(X_test)

# 精度を計算してみましょう。
print(metrics.accuracy_score(Y_test,Y_pred))

In [336]:
Y_test = Series(Y_test)
Y_pred = Series(Y_pred)

In [337]:
hikaku = []

In [338]:
hikaku = pd.concat([Y_test, Y_pred], axis=1)

In [339]:
hikaku

,0,1
0,愛は静けさの中に - みんなのシネマレビュー,愛は静けさの中に - みんなのシネマレビュー
1,アダムス・ファミリー２ - みんなのシネマレビュー,アダムス・ファミリー２ - みんなのシネマレビュー
2,アダムス・ファミリー - みんなのシネマレビュー,アダムス・ファミリー２ - みんなのシネマレビュー
3,アダムス・ファミリー２ - みんなのシネマレビュー,アダムス・ファミリー２ - みんなのシネマレビュー
4,アダムス・ファミリー - みんなのシネマレビュー,アダムス・ファミリー - みんなのシネマレビュー
5,あなたに降る夢 - みんなのシネマレビュー,あなたに降る夢 - みんなのシネマレビュー
6,アダムス・ファミリー - みんなのシネマレビュー,アダムス・ファミリー - みんなのシネマレビュー
7,アダムス・ファミリー２ - みんなのシネマレビュー,アダムス・ファミリー２ - みんなのシネマレビュー
8,あなたに降る夢 - みんなのシネマレビュー,あなたに降る夢 - みんなのシネマレビュー
9,アサシン(1993) - みんなのシネマレビュー,アサシン(1993) - みんなのシネマレビュー
